# Caffe Training Tool

## 1. Import Library

In [ ]:
# Caffe Directory Setting
caffe_root = '../'

import sys 
import os
import os.path as osp
import numpy as np
import codecs
import random

# Import Caffe Python Library
sys.path.append(caffe_root + 'python')
import caffe 

# Import Custom Library
sys.path.append("utils")
sys.path.append("utils/layers")
sys.path.append("utils/nets")

from multilabel_datalayers import *
from pynet import *
import tools

# 2. Setup Parameters

In [ ]:
# Dataset Directory Setting
dataset_path = "../arg-models/text-renderer/output/street_en_ratio"
lexicon_path = "../arg-models/text-renderer/list/street_en_list.txt"

# Model Saving Directory Setting
model_folder = "street_en_ratio_1000/"
if os.path.isdir(model_folder+"/snapshot") is False:
    os.makedirs(model_folder+"/snapshot")

# Network Chosen
network_idx = 1
network_mode = [multilabel_bvlc, multilabel_vgg_dictnet, multilabel_large_vgg, multilabel_vgg16]
network = network_mode[network_idx]

# Training & Testing File
createFile = True

# Hyper-Parameters
policy = "step"
batch_sz = 32
img_shape = [32, 100, 1]
img_channel = 1
    
# Pre-Trained Model
hasPretrained = False
pretrained_model = '../models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel' #model_folder + 'vgg16.caffemodel'

# Caffe Mode Set
caffe.set_mode_gpu()
caffe.set_device(0)

## 3. Load Classes and Lexicon

In [ ]:
pair_list = np.array([line.rstrip("\n").split(",") for line in codecs.open(lexicon_path, 'r', 'utf8').readlines()])

classes = pair_list.flatten() 

distinct_classes = np.array([c for e,c in enumerate(classes) if c not in classes[:e]])
pair_name = pair_list[...,0]
    
print "classes:", distinct_classes
print "-----------------------"
print "name:", pair_name

## 4. Create Training & Testing Data Paths

In [ ]:
if createFile is True:
    
    content = []
    with codecs.open(os.path.join(dataset_path, "path.txt"), 'w', 'utf8') as p:
        print os.path.isdir(dataset_path)
        for dirPath, dirNames, fileNames in os.walk(dataset_path):
            for file in fileNames:
                if file.endswith('.jpg') is False:
                    continue
                path = os.path.join(dirPath, file)
                
                label = unicode(dirPath.split("/")[-1], "utf-8")
                label_idx = np.where(pair_name == label)[0][0]          
                labels = pair_list[label_idx]
                labels_each_num = [str(np.where(distinct_classes == label)[0][0]) for label in labels]            

                out = unicode(path, "utf-8")+","+','.join(labels_each_num)
                content.append(out)
                p.write(out)


    random.shuffle(content)    
    
    with codecs.open(os.path.join(dataset_path, "train.txt"), 'w', 'utf8') as train:
        train.write('\n'.join(content[:int(len(content)*0.9)]))
        
    with codecs.open(os.path.join(dataset_path, "val.txt"), 'w', 'utf8') as val:
        val.write('\n'.join(content[int(len(content)*0.9):]))

    print "File Created!"
    
def getTestLength():
    return len(codecs.open(os.path.join(dataset_path, "path.txt"), "r").readlines())

## 5. Write Prototxt Files

In [ ]:
# Writing Solver Prototxt by tools.py

solverprototxt = tools.CaffeSolver(trainnet_prototxt_path = osp.join(model_folder, "trainnet.prototxt"), testnet_prototxt_path = osp.join(model_folder, "valnet.prototxt"))

solverprototxt.sp['display'] = "50"
solverprototxt.sp['base_lr'] = "0.001"
solverprototxt.sp['snapshot'] = "50"
solverprototxt.sp['test_interval'] = "2000"
solverprototxt.sp['snapshot_prefix'] = "\""+osp.join(model_folder, "snapshot")+"\""

if policy == "step":
    solverprototxt.sp['lr_policy'] = "\"step\""
    solverprototxt.sp['stepsize'] = "100"

solverprototxt.write(osp.join(model_folder, 'solver.prototxt'))

# Writing Trainnet & Valnet Prototxt
with open(osp.join(model_folder, 'trainnet.prototxt'), 'w') as f:
    data_layer_params = dict(batch_size = batch_sz, im_shape = img_shape, split = 'train', data_folder = dataset_path, lexicon = lexicon_path, channel=img_channel)
    f.write(network(data_layer_params, len(distinct_classes)))

with open(osp.join(model_folder, 'valnet.prototxt'), 'w') as f:
    data_layer_params = dict(batch_size = batch_sz, im_shape = img_shape, split = 'val', data_folder = dataset_path, lexicon = lexicon_path, channel=img_channel)
    f.write(network(data_layer_params, len(distinct_classes)))

## 6. Prediction

In [1]:
def check_accuracy_onehot(net, num_batches, batch_size = 128):
    hit = 0.0
    
    for t in range(num_batches):
        net.forward()
        gts = net.blobs['label'].data
        ests = net.blobs['score'].data 
        
        for gt, est in zip(gts, ests): 
            hit += (int(gt) == np.argmax(est))

        if t%5 == 0:

            dt = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print dt, t, "iters, sum of hit:", hit

    return hit / (num_batches * batch_size)

def check_accuracy_multi(net, num_batches, batch_size = 128):
    hit = 0.0
    
    for t in range(num_batches):
        net.forward()
        gts = net.blobs['label'].data
        ests = net.blobs['score'].data  > 0
        
        for gt, est in zip(gts, ests): 
            hit_num = sum([1 for g in gt if g == e])
            hit += (hit_num == len(gt))

        if t%5 == 0:

            dt = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print dt, t, "iters, sum of hit:", hit

    return hit / (num_batches * batch_size)

In [ ]:
solver = caffe.SGDSolver(osp.join(model_folder, 'solver.prototxt'))
solver.net.copy_from(model_name)
solver.test_nets[0].share_with(solver.net)

dt = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print dt, ", predict start!"
check_accuracy_onehot(solver.test_nets[0], testLength/128+1)

BatchLoader initialized with 25325 images